### CPU 환경

In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8153 CPU @ 2.00GHz
Stepping:                        4
CPU MHz:                         2204.039
CPU max MHz:                     2800.0000
CPU min MHz:                     1000.0000
BogoMIPS:                        4000.00
Virtualization:                  VT-x
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        32 MiB
L3 cache:                   

### GPU 환경

In [2]:
!nvidia-smi

Wed Feb 17 07:43:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:37:00.0 Off |                    0 |
| N/A   74C    P0    34W /  70W |   3259MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:86:00.0 Off |                    0 |
| N/A   

### RAM 용량 확인

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:          125Gi       9.3Gi       1.6Gi        50Mi       114Gi       115Gi
Swap:         8.0Gi        86Mi       7.9Gi


### HDD 용량 확인

In [4]:
# 디스크 용량 확인
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev             63G     0   63G   0% /dev
tmpfs            13G  2.7M   13G   1% /run
/dev/sda2       549G  418G  104G  81% /
tmpfs            63G   12K   63G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs            63G     0   63G   0% /sys/fs/cgroup
/dev/sda1       511M  7.8M  504M   2% /boot/efi
/dev/loop4       56M   56M     0 100% /snap/core18/1944
tmpfs            13G     0   13G   0% /run/user/1000
/dev/loop6       32M   32M     0 100% /snap/snapd/10707
/dev/loop0       70M   70M     0 100% /snap/lxd/19032
/dev/loop1       70M   70M     0 100% /snap/lxd/19188
/dev/sdb2       466G  240G  226G  52% /home/test/mount_folder
/dev/loop5       56M   56M     0 100% /snap/core18/1988
/dev/loop2       32M   32M     0 100% /snap/snapd/11036


### OS 환경

In [5]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="20.04.1 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.1 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


## 패키지 불러오기

In [6]:
import sys
import os
import os.path as osp
from glob import glob

import pytz
import datetime
import time

import numpy as np
import pandas as pd
from tqdm import tqdm

import sklearn

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2


import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import explain
import chicken_dataset as Dataset
import Utility_invert as U

In [7]:
print('python version:',sys.version)
print('pandas version:',pd.__version__)
print('numpy version:', np.__version__)

print('sklearn version:', sklearn.__version__)
print('torch version:', torch.__version__)
#print('torchvision version:', torchvision.__version__)
print('albumentations version:', A.__version__)

python version: 3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]
pandas version: 1.1.5
numpy version: 1.19.2
sklearn version: 0.23.2
torch version: 1.7.1
albumentations version: 0.5.2


## 모델 파라미터 (Parameter)

In [8]:
# 모델에 입력할 Input 이미지의 크기를 지정함.
IMG_WIDTH = 448
IMG_HEIGHT = 448

# 배치 사이즈 임의 지정
batch_size = 4

# 데이터를 읽어올 경로를 지정함.
data_dir='../Dataset/preprocessed/classification'
# 학습된 모델이 저장된 경로를 지정함.
ckpt_dir = "./Models/"

# 모델 이름
Model_name = "chicken_classification"
# 모델을 연산할 장비 설정 (Docker환경에서 작동할 수 있도록 CPU에서 계산함)
device = torch.device('cpu')

## Train, Validation, Test 데이터 셋 확인

Abnormal의 경우, 총 85개 이미지가 있으며 train : valid : test = 55 : 15 : 15 비율을 따랐음. (Preprocess.ipynb 참고)<br>
이를 바탕으로, valid, test set 안에서 Abnormal 이 10% 를 차지하도록 구성하였음.<br>
("Deep Learning for Anomaly Detection: A Review", 2020, Pang. G, Shen. C, et al. 의 Table 3 을 참고하여 10% 로 선정.)

train set의 class 분포는 위와 같은 비율을 따르지 않고 Normal의 모든 이미지를 사용함.<br>
또, 55개의 Abnormal 데이터를 20배 가량 단순 over-sampling 하여, 더 나은 모델 학습을 기대함.

## 데이터 셋 (Dataset) 불러오기

In [9]:
# 이미지 transformation
test_compose=A.Compose([
    A.Resize(IMG_HEIGHT,IMG_WIDTH),
    ToTensorV2()
])

# Test set을 읽어옴
test_dataset=Dataset.ChickenDataset(os.path.join(data_dir, 'test'), transforms=test_compose)
test_dataloader=DataLoader(test_dataset, batch_size=batch_size, pin_memory=False, shuffle=False)

In [10]:
# criterions : 파계 분류 기준 설정에 필요한 metric
criterions = {
"Reconstruction": nn.MSELoss(reduction='none')
}    

### 모델 불러오기 (Load)

In [11]:
# 학습된 모델 가중치를 불러옴
checkpoint=torch.load(ckpt_dir + Model_name + '.pth', map_location=device)

# model : Adversarial AutoEncoder
models=U.Create_Models(base_model='RESNET50')

models['Encoder'].load_state_dict(checkpoint['state_dict']['Encoder'])
models['Decoder'].load_state_dict(checkpoint['state_dict']['Decoder'])
threshold=checkpoint['val_best_f1_threshold']

<All keys matched successfully>

<All keys matched successfully>

### 모델 학습 전, ImageNet 데이터로 Pretrain 된 기본 모델 불러오기 (Load)

In [12]:
# model : Adversarial AutoEncoder
base_model=U.Create_Models(base_model='RESNET50')

## 모델 확인 (Summary)

In [13]:
models

{'Encoder': Encoder(
   (model_ft): ResNet(
     (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
     (layer1): Sequential(
       (0): Bottleneck(
         (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
         (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
         (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
         (relu): ReLU(inplace=True)
         (downsam

---

## 테스트 (Test) 수행

### 1. ImageNet 데이터로 Pretrain 된 기본 모델 성능 평가

### 테스트 이미지 평가 (Predict)

In [14]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-17 16:44:00


In [15]:
since=time.time()

In [16]:
# 네트워크를 evaluation 용으로 선언
for key in list(base_model.keys()):
    if key != "Z_dim":
        base_model[key]=base_model[key].eval().to(device)
        
# 모델 성능 확인을 위한 결과물 저장
recon_dict={'recon_error':[], 'true_label_tot': []}

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for i, (images, target) in tqdm(enumerate(test_dataloader)):
        
        images=images.to(device)
        target=target.to(device)

        recon_dict['true_label_tot'].extend(list(target.numpy()))

        x, z = base_model["Encoder"](images)
        x_recon = base_model["Decoder"](z)
        
        reconstruction_loss = criterions["Reconstruction"](x_recon, x).mean(dim=1)            
        recon_dict['recon_error'].extend(reconstruction_loss.detach().cpu().numpy().tolist())


total_recon_loss, true_label_tot= recon_dict['recon_error'] , recon_dict['true_label_tot']

38it [01:41,  2.68s/it]


In [17]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 1m 41s


In [18]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-17 16:45:42


## F1-Score

In [19]:
# Confusion Matrix 생성
prediction=explain.Prediction(total_recon_loss, true_label_tot, threshold)
true_total_label, pred_total_label=prediction.get_prediction()

In [20]:
result=explain.ShowResult(true_total_label, pred_total_label)
result.show_result()

Predicted  Abnormal  All
True                    
Normal          135  135
Abnormal         15   15
All             150  150


#-- Confusion Matrix for class Normal

                      Pred       
                Non Normal Normal
True Non Normal         15      0
     Normal            135      0
F1-Score for class Normal : 0.000
-----------------------------------

#-- Confusion Matrix for class Abnormal

                          Pred         
                  Non Abnormal Abnormal
True Non Abnormal            0      135
     Abnormal                0       15
F1-Score for class Abnormal : 0.182
-----------------------------------

#-- Final Macro F1-Score
( 0.000 + 0.182 ) / 2 = 0.0909


---

### 2. 기본 모델을 Fine Tuning (학습) 한 모델 성능 평가 (Transfer Learning)

### 테스트 이미지 평가 (Predict)

In [21]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 시작 시간 : {nowDatetime}')

테스트 시작 시간 : 2021-02-17 16:45:43


In [22]:
since=time.time()

In [23]:
# 네트워크를 evaluation 용으로 선언
for key in list(models.keys()):
    if key != "Z_dim":
        models[key]=models[key].eval().to(device)
        
# 모델 성능 확인을 위한 결과물 저장
recon_dict={'recon_error':[], 'true_label_tot': []}

# test 이기 때문에 backprop 진행 x
with torch.no_grad():
    for i, (images, target) in tqdm(enumerate(test_dataloader)):
        
        images=images.to(device)
        target=target.to(device)

        recon_dict['true_label_tot'].extend(list(target.numpy()))

        x, z = models["Encoder"](images)
        x_recon = models["Decoder"](z)
        
        reconstruction_loss = criterions["Reconstruction"](x_recon, x).mean(dim=1)            
        recon_dict['recon_error'].extend(reconstruction_loss.detach().cpu().numpy().tolist())


total_recon_loss, true_label_tot= recon_dict['recon_error'] , recon_dict['true_label_tot']

38it [01:46,  2.80s/it]


In [24]:
end=time.time()
print(f'테스트 소요 시간: {int((end-since)//60)}m {int((end-since)%60)}s')

테스트 소요 시간: 1m 46s


In [25]:
now = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
print(f'테스트 종료 시간 : {nowDatetime}')

테스트 종료 시간 : 2021-02-17 16:47:29


## F1-Score

In [26]:
# Confusion Matrix 생성
prediction=explain.Prediction(total_recon_loss, true_label_tot, threshold)
true_total_label, pred_total_label=prediction.get_prediction()

In [27]:
result=explain.ShowResult(true_total_label, pred_total_label)
result.show_result()

Predicted  Normal  Abnormal  All
True                            
Normal        131         4  135
Abnormal        5        10   15
All           136        14  150


#-- Confusion Matrix for class Normal

                      Pred       
                Non Normal Normal
True Non Normal         10      5
     Normal              4    131
F1-Score for class Normal : 0.967
-----------------------------------

#-- Confusion Matrix for class Abnormal

                          Pred         
                  Non Abnormal Abnormal
True Non Abnormal          131        4
     Abnormal                5       10
F1-Score for class Abnormal : 0.690
-----------------------------------

#-- Final Macro F1-Score
( 0.967 + 0.690 ) / 2 = 0.8282
